In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
def trainmerge(trainZ,mergeB,onvar):
    return trainZ.merge(pd.read_csv('../input/ashrae-energy-prediction/'+mergeB+'.csv'), on=onvar, how='left')

def loaddata(trainM,xi,splitten):
    if splitten:
        trainX=trainmerge(trainmerge(pd.read_csv('../input/ashrae-energy-prediction/'+trainM+'.csv')[xi*11000000:(xi+1)*11000000],'building_metadata','building_id'),'weather_'+trainM,['site_id', 'timestamp'])
    else:
        trainX=trainmerge(trainmerge(pd.read_csv('../input/ashrae-energy-prediction/'+trainM+'.csv'),'building_metadata','building_id'),'weather_'+trainM,['site_id', 'timestamp'])
    
    print(trainX.info())
    numericcolumns=['air_temperature','cloud_coverage','dew_temperature','floor_count','meter','precip_depth_1_hr','sea_level_pressure','wind_direction','wind_speed','year_built']

    return trainX


train=loaddata('train',0,False)

In [ ]:
EnergypUse=train[['primary_use','meter_reading']].groupby('primary_use').mean().sort_values('meter_reading')

In [ ]:
#todo split date in month

def HotEnc(totaal,codes,indexv,label,trainvw):
    #add variables
    totaal['M2K']=totaal['square_feet'].values*(25-totaal['air_temperature'].values)
    totaal['pM2K']=1/totaal['M2K']
    if trainvw:
        totaal['EpM2']=totaal['meter_reading'].fillna(0)/totaal['square_feet'].values  #energy use is function of surface...
        #totaal['EpM2K']=totaal['meter_reading'].fillna(0)*totaal['pM2K']
        #totaal['EpM2K']=totaal['EpM2K'].fillna(0).replace(np.inf,0)
        #totaal['EpM2K']=totaal['EpM2K'].astype('int')
    totaal['floor_count'] = totaal['floor_count'].fillna(0).astype(np.int8)
    totaal['year_built'] = totaal['year_built'].fillna(0).astype(np.int16)

    totaal['age'] = (2019-totaal['year_built'].fillna(totaal['year_built'].median())).astype(np.int8)
    totaal["timestamp"] = pd.to_datetime(totaal["timestamp"], format='%Y-%m-%d %H:%M:%S')
    totaal['month_datetime'] = totaal['timestamp'].dt.month.astype(np.int8)
    totaal['weekofyear_datetime'] = totaal['timestamp'].dt.weekofyear.astype(np.int8)
    totaal['dayofyear_datetime'] = totaal['timestamp'].dt.dayofyear.astype(np.int16)
    totaal['hour_datetime'] = totaal['timestamp'].dt.hour.astype(np.int8)  
    totaal['day_week'] = totaal['timestamp'].dt.dayofweek.astype(np.int8)
    totaal['day_month_datetime'] = totaal['timestamp'].dt.day.astype(np.int8)
    totaal['year_built'] = totaal['year_built']-1900
    totaal['square_feet'] = np.log(totaal['square_feet'])
     
    # One Hot Encode target mean()
    codesdict=codes[label].to_dict()
    totaal['primary_use']=totaal['primary_use'].map(codesdict)
    
    print(totaal.info())
    return totaal.drop(['timestamp','precip_depth_1_hr'],axis=1)

train=HotEnc(train,EnergypUse,'leeg','meter_reading',True)
#test=loaddata('test')
#test=HotEnc(test,train,'leeg','meter_reading',False)

In [ ]:
train

In [ ]:
def clustertechniques2(dtrain,label,indexv):
    print('#encodings',dtrain.shape)
    cols=[ci for ci in dtrain.columns if ci not in [indexv,'index',label]]
    dtest=dtrain[dtrain[label].isnull()==True][[indexv,label]]
    print(dtest)

    print('encodings  after shape',dtrain.shape)
    #split data or use splitted data
    X_train=dtrain[dtrain[label].isnull()==False].drop([indexv,label],axis=1).fillna(0)
    Y_train=dtrain[dtrain[label].isnull()==False][label]
    X_test=dtrain[dtrain[label].isnull()==True].drop([indexv,label],axis=1).fillna(0)
    Y_test=np.random.random((X_test.shape[0],1))
    if len(X_test)==0:
        from sklearn.model_selection import train_test_split
        X_train,X_test,Y_train,Y_test = train_test_split(dtrain.drop(label,axis=1).fillna(0),dtrain[label],test_size=0.25,random_state=0)
    lenxtr=len(X_train)
    print('splitting data train test X-y',X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)
   


    import matplotlib.pyplot as plt 
    from sklearn import preprocessing
    scale = preprocessing.MinMaxScaler().fit(X_train)
    X_train = scale.transform(X_train)
    X_test = scale.transform(X_test)

        
    from sklearn.decomposition import PCA,TruncatedSVD,NMF,FastICA
    from umap import UMAP  # knn lookalike of tSNE but faster, so scales up
    from sklearn.manifold import TSNE #limit number of records to 100000

    clusters = [Dummy(1),
                PCA(n_components=0.7,random_state=0,whiten=True),
                #FastICA(n_components=7,random_state=0),
                TruncatedSVD(n_components=10, n_iter=7, random_state=42),
                #NMF(n_components=10,random_state=0),            
                #UMAP(n_neighbors=5,n_components=10, min_dist=0.3,metric='minkowski'),
                #TSNE(n_components=2,random_state=0)
                ] 
    clunaam=["raw",'PCA','tSVD','UMAP']#,'ICA','tSVD','nmf','UMAP','tSNE']
    
    
    from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
    from sklearn.svm import SVC, LinearSVC,NuSVC
    from sklearn.multiclass import OneVsRestClassifier
    from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor, RandomForestClassifier,ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
    from sklearn.neural_network import MLPClassifier,MLPRegressor
    from sklearn.linear_model import PassiveAggressiveClassifier,Perceptron,SGDClassifier,LogisticRegression
    import xgboost as xgb
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
    from sklearn.linear_model import ElasticNetCV,ridge_regression,HuberRegressor,LinearRegression,BayesianRidge,RANSACRegressor
    from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
    from sklearn.metrics import mean_squared_error,r2_score
    classifiers = [

                   #GradientBoostingRegressor(),
                   ExtraTreesRegressor(),
                   RandomForestRegressor(random_state=1, n_estimators=10),        
                   BayesianRidge(),
        
                   RANSACRegressor(),
                   KNeighborsRegressor(),
                   ElasticNetCV(cv=5, random_state=0),
                   HuberRegressor(),
                   LinearRegression(),
                  ]
    clanaam= ['xTreer','rFor','BaysR','Ransac','KNNr','elast','huber','linear',]
    from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
    
    results=[]


    #cluster data
    for clu in clusters:
        clunm=clunaam[clusters.index(clu)] #find naam
        X_total_clu = clu.fit_transform(np.concatenate( (X_train,X_test),axis=0))
        X_total_clu=np.concatenate((X_total_clu,np.concatenate( (X_train,X_test),axis=0)),axis=1)
        print(X_total_clu.shape)
        plt.scatter(X_total_clu[:lenxtr,0],X_total_clu[:lenxtr,1],c=Y_train.values,cmap='prism')
        plt.title(clu)
        plt.show()
        
        #classifiy 
        for cla in classifiers:
            import datetime
            start = datetime.datetime.now()
            clanm=clanaam[classifiers.index(cla)] #find naam
            
            print('    ',cla)
            cla.fit(X_total_clu,np.concatenate( (Y_train,Y_test)) )
            cla.fit(X_total_clu[:lenxtr],Y_train )
            
            #predict
            trainpredi=cla.predict(X_total_clu[:lenxtr])

            #print(classification_report(trainpredi,Y_train))            
            testpredi=cla.predict(X_total_clu[lenxtr:])  
            if classifiers.index(cla) in [0,2,3,4,5,7,8,9,10,11,12,13]:
                trainprediprob=cla.predict(X_total_clu[:lenxtr])
                testprediprob=cla.predict(X_total_clu[lenxtr:]) 
                
                plt.scatter(x=testprediprob, y=testpredi, marker='.', alpha=0.53)
                plt.show()            
            #testpredi=converging(pd.DataFrame(X_train),pd.DataFrame(X_test),Y_train,pd.DataFrame(testpredi),Y_test,clu,cla) #PCA(n_components=10,random_state=0,whiten=True),MLPClassifier(alpha=0.510,activation='logistic'))
            
            if len(dtest)==0:
                test_score=cla.score(X_total_clu[lenxtr:],Y_test)
                mse = mean_squared_error(testpredi,Y_test)
                train_score=cla.score(X_total_clu[:lenxtr],Y_train)

                li = [clunm,clanm,train_score,test_score,mse]
                results.append(li)
                r2s=r2_score(testpredi,Y_test)  
                print(r2s)

                plt.title(clanm+'test mse versus unknown:'+np.str(test_score)+' '+np.str(mse)+' and test confusionmatrix')
                plt.scatter(x=Y_test, y=testpredi, marker='.', alpha=1)
                plt.scatter(x=[np.mean(Y_test)], y=[np.mean(testpredi)], marker='o', color='red')
                plt.xlabel('Real test'); plt.ylabel('Pred. test')
                plt.show()


            else:
#                testpredlabel=le.inverse_transform(testpredi)  #use if you labellezid the classes 
                testpredlabel=testpredi
                print(confusion_matrix(trainpredi,Y_train))
                submit = pd.DataFrame({indexv: dtest[indexv],label: testpredlabel})
                submit[label]=submit[label].astype('int')

                filenaam='subm_'+clunm+'_'+clanm+'.csv'
                submit.to_csv(path_or_buf =filenaam, index=False)
                
            print(clanm,'0 classifier time',datetime.datetime.now()-start)
            
    if len(dtest)==0:       
        print(pd.DataFrame(results).sort_values(3))
        submit=[]
    return submit

#Custom Transformer that extracts columns passed as argument to its constructor 
class Dummy( ):
    #Class Constructor 
    def __init__( self, feature_names ):
        self._feature_names = feature_names 
    
    #Return self nothing else to do here    
    def fit( self, X, y = None ):
        return self 
    
    #Method that describes what we need this transformer to do
    def fit_transform( self, X, y = None ):
        return X 

clustertechniques2(train[:10000].drop(['meter_reading'],axis=1).fillna(0).replace(np.inf,0),'EpM2','building_id') #total[len(train):].fillna(0

In [ ]:
%%time
from sklearn.ensemble import ExtraTreesRegressor
cla=ExtraTreesRegressor()
cla.fit(train.drop(['EpM2','meter_reading'],axis=1).fillna(0).replace(np.inf,0).values,train[['EpM2']].fillna(0).values)
predicts=[]



In [ ]:
del train

In [ ]:
predicts=pd.DataFrame([])
for xi in range(4):
    #fit regression in two chunck !
    test=loaddata('test',xi,True)
    test=HotEnc(test,EnergypUse,'leeg','meter_reading',False)
    preds=test[['row_id','square_feet']]
    preds['meter']=0.0
    preds['meter'] = cla.predict(test.drop(['row_id'],axis=1).fillna(0).replace(np.inf,0).values)
    preds['meter_reading']=np.exp(preds['square_feet'])*preds['meter']
    predicts=predicts.append(preds)

In [ ]:
predicts[['row_id','meter_reading']].to_csv('xtree.csv', index=False)